In [ ]:
# これまでと同等のMNIST手書き数字認識モデルを構築する
# ただし学習は行わない
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten

img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)
num_classes = 10

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(),
              metrics=['accuracy'])

keras_model_path = './KerasMnistNotTrained.h5'
model.save(keras_model_path)

In [ ]:
## Core MLモデルに変換
from keras.models import load_model
keras_model = load_model(keras_model_path)

from coremltools.converters import keras as converter

class_labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

mlmodel = converter.convert(keras_model,
                            input_names=['image'],
                            output_names=['digitProbabilities'],
                            class_labels=class_labels,
                            predicted_feature_name='digit')

In [ ]:
# 入力の型を28x28のグレースケール画像に変更
import coremltools

spec = mlmodel.get_spec()
builder = coremltools.models.neural_network.NeuralNetworkBuilder(spec=spec)

from coremltools.proto import FeatureTypes_pb2 as ft
grayscale = ft.ImageFeatureType.ColorSpace.Value('GRAYSCALE')
input_image_type = builder.spec.description.input[0].type.imageType
input_image_type.width = 28
input_image_type.height = 28
input_image_type.colorSpace = grayscale

In [ ]:
print(spec.isUpdatable)
print(spec.specificationVersion)

In [ ]:
builder.inspect_updatable_layers()

In [ ]:
builder.inspect_layers()

In [ ]:
# モデルをUpdatableにする
builder.make_updatable(['conv2d_1', 'conv2d_2', 'dense_1', 'dense_2'])

In [ ]:
# spec.isUpdatable = True
# spec.specificationVersion = coremltools._MINIMUM_UPDATABLE_SPEC_VERSION
# は不要（make_updatableの中でやってくれている）
print(spec.isUpdatable)
print(spec.specificationVersion)

In [ ]:
builder.inspect_updatable_layers()

In [ ]:
builder.inspect_layers()

In [ ]:
# 損失関数を指定する
builder.set_categorical_cross_entropy_loss(name='lossLayer', input='digitProbabilities')

In [ ]:
builder.inspect_loss_layers()

In [ ]:
# 最適化アルゴリズム（オプティマイザ）を指定する
from coremltools.models.neural_network import SgdParams
builder.set_sgd_optimizer(SgdParams(lr=0.001, batch=8))

In [ ]:
builder.inspect_optimizer()

In [ ]:
# エポック数を指定する
builder.set_epochs(12)

In [ ]:
# 保存
from coremltools.models import MLModel

coreml_updatable_model_path = './UpdatableMNISTDigitClassifier.mlmodel'
mlmodel_updatable = MLModel(spec)
mlmodel_updatable.save(coreml_updatable_model_path)